## Hypothesentests für den Mittelwert einer Grundgesamtheit, wenn Sigma unbekannt ist
----------------------------------------

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import sample
from scipy.stats import t
from scipy import stats

%run ../src/notebook_env.py


---------------------------------
Working on the host: LAPTOP-9LETB4SJ

---------------------------------
Python version: 3.10.2 | packaged by conda-forge | (main, Mar  8 2022, 15:52:24) [MSC v.1929 64 bit (AMD64)]

---------------------------------
Python interpreter: C:\Users\zak\anaconda3\envs\srh\python.exe


Ein Hypothesentest für einen Grundgesamtheitsmittelwert, bei dem die Standardabweichung $\sigma$ der Grundgesamtheit nicht bekannt ist, wird auf die gleiche Weise durchgeführt, wie wenn die Standardabweichung der Grundgesamtheit bekannt ist. Der einzige Unterschied besteht darin, dass die <a href="https://de.wikipedia.org/wiki/Studentsche_t-Verteilung">$t$-Verteilung</a> und nicht die Standardnormalverteilung ($z$-Verteilung) verwendet wird.

Bei einem Test mit der Nullhypothese $H_0:\; \mu = \mu_0$ wird die Teststatistik, $t$, wie folgt berechnet

$$t = \frac{\bar x - \mu_0}{s/\sqrt{n}}$$

Dieses Hypothesentestverfahren wird als **$t$-Test für einen Mittelwert** oder einfach als **$t$-Test** bezeichnet. Es sei daran erinnert, dass Hypothesentests einem schrittweisen Verfahren folgen, das wie folgt zusammengefasst wird 

$$
\begin{array}{l}
\hline
\ \text{Schritt 1}  & \text{Geben Sie die Nullhypothese } H_0 \text{ und alternative Hypothese } H_A \text{ an.}\\
\ \text{Schritt 2}  & \text{Legen Sie das Signifikanzniveau, } \alpha\text{ fest.} \\
\ \text{Schritt 3}  & \text{Berechnen Sie den Wert der Teststatistik.} \\
\ \text{Schritt 4a} & \text{Ansatz des kritischen Wertes: Bestimmung des kritischen Wertes.}
\\
\ \text{Schritt 4b} &\text{P-Wert-Ansatz: Bestimmen Sie den p-Wert.} \\
\ \text{Schritt 5a} & \text{Ansatz des kritischen Werts: Wenn der Wert der Teststatistik in den Ablehnungsbereich fällt, ist } H_0 \text{ abzulehnen}   \text{; ansonsten } H_0 \text{ nicht ablehnen}  \text{.}
\\
\ \text{Schritt 5b} & \text{P-Wert-Ansatz: Wenn }p \le \alpha \text{, } H_0 \text{ ablehnen } \text{; ansonsten } H_0 \text{ nicht ablehnen} \text{.} \\
\ \text{Schritt 6} &\text{Interpretieren Sie das Ergebnis des Hypothesentests.} \\
\hline 
\end{array}
$$

Ähnlich wie im vorangegangenen Abschnitt stellen wir zunächst den Ansatz des **kritischen Werts** vor und wiederholen dann in einem zweiten Schritt die Analyse für den **$p$-Wert-Ansatz**. Diesmal verpacken wir jedoch den Ansatz des kritischen Werts in eine selbst erstellte Funktion. Für den $p$-Wert-Ansatz werden wir die leistungsstarke Maschinerie von Python nutzen und die vorhandene Funktion `ttest_ind` anwenden.

### Hypothesentests: Der Ansatz des kritischen Werts

Erstellen wir eine Funktion namens `simple_ttest()`. Die Funktion nimmt als Eingangsargumente einen Vektor `x` (Stichprobendaten), `mu0`, das $\mu_0$ entspricht, den Grundgesamtheitsdurchschnitt, gegen den getestet werden soll, das angegebene Signifikanzniveau $\alpha$, bezeichnet als `alpha`, und die Methode des $t$-Tests, `links`, `rechts` oder der Standardwert `zweiseitig`, mit dem Funktionsargument namens `Methode`. Die Ausgabe der Funktion ist ein Boolescher Wert, `TRUE` oder `FALSE`. Wenn `TRUE`, wird $H_0$ abgelehnt, wenn `FALSE`, wird $H_0$ nicht abgelehnt.

In [20]:
def simple_ttest(x, mu0, alpha, method='two-sided'):
    n = len(x)
    xbar = np.mean(x)
    s = np.std(x,ddof = 1)
    # calculate test statistic
    tstat = (xbar-mu0)/(s/np.sqrt(n))
    
    # calculate critical value
    df = n-1
    # for left-tailed test
    if (method=='left'):
        crit_val = t.ppf(q = alpha, df = df)
          #evaluate rejection region
        if (tstat < crit_val):
            reject = True
        else:
            reject = False
        # for right-tailed test
    if method=='right':
            crit_val = t.ppf(q = 1-alpha, df = df)
          #evaluate rejection region
            if (tstat > crit_val):
                reject = True
            else:
                reject = False
          
        # for two-sided test (default)
    if method=='two-sided':
            crit_val = t.ppf(q = 1-alpha/2, df = df)
        #evaluate rejection region
            if (abs(tstat) > abs(crit_val) and -abs(tstat) < -abs(crit_val)):
                reject = True
            else:
                reject = False    

            
    print('Significance level:',alpha)
    print('Degrees of freedom:',df)
    print('Test statistic:',round(tstat,4))
    print('Critical value:',round(crit_val,4))
    print('Reject H0:',reject)       
          


Ein tolles Stück Code :-)

### $t$-Test mit einem Mittelwert: Ein Beispiel

Nun ist es an der Zeit, unsere Funktion `simple_ttest()` zu testen. Dazu wiederholen wir das Beispiel aus dem vorherigen Abschnitt. Wir verwenden den `students` Datensatz. Sie können die Datei `students.csv` <a href="https://userpage.fu-berlin.de/soga/200/2010_data_sets/students.csv">hier</a> herunterladen. Importieren Sie den Datensatz und geben Sie ihm einen passenden Namen.

In [21]:
# Lese Datei students.csv als Dataframe ein; Indexspalte wird übersprungen
students = pd.read_csv('students.csv', index_col=0)

Der `students` Datensatz besteht aus $8239$ Zeilen, von denen jede einen bestimmten Studenten repräsentiert, und $16$ Spalten, von denen jede einer Variable/einem Merkmal entspricht, das sich auf diesen bestimmten Studenten bezieht. Diese selbsterklärenden Variablen sind: *stud.id, name, gender, age, height, weight, religion, nc.score, semester, major, minor, score1, score2, online.tutorial, graduated, salary.*

Wir untersuchen das Durchschnittsgewicht einer Zufallsstichprobe von Studenten aus dem `students` Datensatz und vergleichen es mit dem Durchschnittsgewicht aller erwachsenen Europäer. <a href="https://bmcpublichealth.biomedcentral.com/articles/10.1186/1471-2458-12-439">Walpole et al. (2012)</a> veröffentlichten Daten über das durchschnittliche Körpergewicht (kg) pro Region, einschließlich Europa. Sie geben die durchschnittliche Körpermasse für die europäische erwachsene Bevölkerung mit $70,8$ kg an. Wir setzen daher $μ_0$, den Mittelwert der Grundgesamtheit, entsprechend fest ($μ_0=70,8$). Ferner ziehen wir eine Zufallsstichprobe ($x$) mit einem Umfang von $n=9$. Die Stichprobe besteht aus den Gewichten in kg von $9$ zufällig ausgewählten Studenten.

In [22]:
mu0 = 70.8
n = 9
#weight=list(students['weight'])
x = students['weight'].sample(n,random_state = 1)
x.head(10)

7005    56.5
3206    86.5
4544    62.9
2104    62.4
3510    64.4
628     76.3
537     65.7
5419    61.4
645     66.6
Name: weight, dtype: float64

### Überprüfen der Hypothesen

**Schritt 1 : Geben Sie die Nullhypothese $H_0$ und alternative Hypothese $H_A$ an**

Die Nullhypothese besagt, dass das Durchschnittsgewicht der Studenten dem Durchschnittsgewicht der europäischen Erwachsenen entspricht, wie von Walpole et al. (2012) berichtet. Mit anderen Worten: Es gibt keinen Unterschied zwischen dem Durchschnittsgewicht der Studenten und dem Durchschnittsgewicht der europäischen Erwachsenen.

$$H_0: \quad \mu = 70,8$$

Erinnern Sie sich daran, dass die Formulierung der Alternativhypothese vorgibt, ob wir einen zweiseitigen, einen links- oder einen rechtsseitigen Hypothesentest durchführen.

**Alternativhypothese 1**

$$H_{A_1}: \quad \mu \ne 70,8$$

führt zu einem zweiseitigen Hypothesentest.

**Alternativhypothese 2**

$$H_{A_2}: \quad \mu  < 70,8$$

zu einem linksseitigen Hypothesentest und

**Alternativhypothese 3**

$$H_{A_3}: \quad \mu > 70,8$$

zu einem rechtsseitigen Hypothesentest

**Schritt 2: Legen Sie das Signifikanzniveau,$\alpha$ fest.**

$$\alpha = 0,05$$

In [23]:
alpha = 0.05

**Schritt 3, 4 und 5: Berechnen Sie den Wert der Teststatistik, bestimmen Sie den kritischen Wert, und werten Sie den Wert der Teststatistik aus. Wenn er in den Verwerfungsbereich fällt, verwerfen Sie $H_0$; andernfalls verwerfen Sie $H_0$ nicht**

Jetzt kommt unsere selbst erstellte Funktion `simple_ttest()` ins Spiel. Wir geben der Funktion eine Zufallsstichprobe in Form eines Vektors, einen Wert für $\mu_0$, ein Signifikanzniveau $\alpha$ und die Methode (`two-sided`, `left` oder `right`) an. Erinnern Sie sich daran, dass zweiseitig der Standardwert ist. Wenn wir also keine Methode angeben, wird die Funktion den zweiseitigen Hypothesentest anwenden.

In [24]:
simple_ttest(x, mu0, alpha)

Significance level: 0.05
Degrees of freedom: 8
Test statistic: -1.271
Critical value: 2.306
Reject H0: False


Cool! ;-)

Ein rechtsseitiger $t$-Test

In [25]:
simple_ttest(x, mu0, alpha, method = 'right')

Significance level: 0.05
Degrees of freedom: 8
Test statistic: -1.271
Critical value: 1.8595
Reject H0: False


Ein linksseitiger $t$-Test

In [26]:
simple_ttest(x, mu0, alpha, method = 'left')

Significance level: 0.05
Degrees of freedom: 8
Test statistic: -1.271
Critical value: -1.8595
Reject H0: False


**Schritt 6: Interpretieren Sie das Ergebnis des Hypothesentests**

Wenn die Teststatistik und damit der Stichprobenmittelwert so extrem ist, dass er über den kritischen Wert hinausgeht und damit in den Verwerfungsbereich fällt, schließen wir, dass die Daten bei einem Signifikanzniveau von $5 \%$ genügend Beweise liefern, um $H_0$ zu verwerfen. Fällt die Teststatistik und damit der Stichprobenmittelwert dagegen in den Nicht-Verwerfungsbereich, so schließen wir, dass die Daten keine Beweise für die Ablehnung von $H_0$ liefern.

### Hypothesentests in Python: Der $p$-Wert Ansatz

Der zweite Ansatz basiert auf der Zuweisung einer Wahrscheinlichkeit für den Wert der Teststatistik. Wenn die Teststatistik sehr extrem ist und die Nullhypothese wahr ist, wird der Teststatistik eine geringe Wahrscheinlichkeit zugewiesen. Ist die Teststatistik dagegen überhaupt nicht extrem, so wird ihr eine viel höhere Wahrscheinlichkeit zugewiesen. Diese Wahrscheinlichkeit wird als $p$-Wert bezeichnet.

Um den genauen $p$-Wert für einen gegebenen numerischen Wert zu berechnen, sind wir auf Software angewiesen, da die "In-Tabellen-nachschlagen-Methode" etwas mühsam ist, da nicht alle numerischen Werte der Teststatistik in einer solchen Tabelle angegeben sind. Dies führt zu Rundungsfehlern. Die Programmiersprache Python bietet jedoch einen sehr leistungsfähigen Mechanismus zur Durchführung von $t$-Tests. Die generische Funktion heißt `ttest_1samp`. Sie können mehr Details erfahren, indem Sie `help(stats.ttest_1samp)` in Ihre Konsole eingeben. Obwohl die Funktion verschiedene Argumente liefert, ist der spezielle $t$-Test, für den wir uns jetzt interessieren, nämlich der Test eines Grundgesamtheitsmittelwertes, wenn $\sigma$ unbekannt ist, ist recht einfach.

Wir wiederholen das gleiche Problem wie im vorherigen Abschnitt, wobei wir die bereits definierten Variablen verwenden. Zur Erinnerung: Die Stichprobe ist ein Vektor von Gewichten ($x$), $\mu_0=70,8$, angegeben als `mu0`, $n=9$, angegeben als `n`. Mit diesen Informationen wenden wir die Funktion `ttest_1samp` für einen zweiseitigen $t$-Test an, indem wir `ttest_1samp(x = x, mu = mu0, alternative = 'two-sided')` in Python eingeben. Wir vergleichen zur Auswertung ob der $p$-Wert unter $\alpha=0,05$ liegt und damit statistisch Signifikant ist.

In [18]:
stats.ttest_1samp(x,mu0,alternative='two-sided')

Ttest_1sampResult(statistic=-1.2710092384447174, pvalue=0.23943518262349403)

In [19]:
0.2394<alpha

False

Wahnsinn! Python führt einen Hypothesentest in einer einzigen Codezeile durch und gibt zusätzliche Informationen kostenlos aus! Gehen wir die Ausgabe der Funktion `ttest_1samp()` durch.

Die erste Zeile listet die Ergebnisse der Berechnungen für die Teststatistik $t$ und den $p$-Wert auf: $t = -1,271$, $p$-Wert $= 0,2394$.
    Wir können die Alternativhypothese, die wir verwenden durch das Argument `alternative` = `less` , `greater` , `two-sided` ändern. In unserem Fall war sie zweiseitig. Versuchen Sie selbst, das Attribut in `less` oder `greater` zu ändern, um eine andere Alternativhypothese anzuwenden.

Es ist erwähnenswert, dass die Funktion `ttest_1samp()` die Ergebnisse des $t$-Tests ausgibt. Wir können also die Ergebnisse der Funktion `ttest_1samp()` in einer Variablen speichern.

Um den $p$-Wert abzurufen, geben wir ein

In [27]:
statistics,pvalue=stats.ttest_1samp(x,mu0,alternative='two-sided')

In [28]:
pvalue

0.23943518262349403

In [29]:
statistics

-1.2710092384447174